In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import math
import os

In [2]:
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1   = nn.Conv2d(in_planes, in_planes // 16, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv2d(in_planes // 16, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out		# 这里并没有使用到论文中的shared MLP, 而是直接相加了
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

In [3]:
class SEModule(nn.Module):
    expansion = 1

    def __init__(self, inplanes, stride=1, downsample=None):
        super(SEModule, self).__init__()
        self.conv1 = conv3x3(inplanes, inplanes, stride)
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(inplanes, inplanes)
        self.bn2 = nn.BatchNorm2d(inplanes)

        self.ca = ChannelAttention(inplanes)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = self.ca(out) * out
       
        out += residual
      
#         print(out.size())
        out = self.sa(out) * out
#         print(out.size())
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
#         print(out.size())
        out = self.relu(out)

        return out

In [4]:
def conv31(ip,op,stride=1):
    return nn.Conv2d(ip,op,kernel_size=3,stride=stride,padding=1,bias=False)
def conv32(ip,op,stride=2):
    return nn.Conv2d(ip,op,kernel_size=3,stride=stride,padding=1,bias=False)
class Upsample(nn.Module):
    def __init__(self,ip,op):
        super(Upsample,self).__init__()
        self.upsample=nn.Sequential(
            conv31(ip,op,1),
            nn.Upsample(scale_factor=2,mode='nearest')
        )
        
    def forward(self,x):
        x=self.upsample(x)
        return x

In [5]:
def Hswish(x,inplace=True):
    return x * F.relu6(x + 3., inplace=inplace) / 6.

def Hsigmoid(x,inplace=True):
    return F.relu6(x + 3., inplace=inplace) / 6.


class Bottleneck(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,exp_channels,stride,se='True',nl='HS'):
        super(Bottleneck, self).__init__()
        padding = (kernel_size - 1) // 2
        if nl == 'RE':
            self.nlin_layer = F.relu6
        elif nl == 'HS':
            self.nlin_layer = Hswish
        self.stride=stride
        if se:
            self.se=SEModule(exp_channels)
        else:
            self.se=None
        self.conv1=nn.Conv2d(in_channels,exp_channels,kernel_size=1,stride=1,padding=0,bias=False)
        self.bn1 = nn.BatchNorm2d(exp_channels)
        self.conv2=nn.Conv2d(exp_channels,exp_channels,kernel_size=kernel_size,stride=stride,
                             padding=padding,groups=exp_channels,bias=False)
        self.bn2=nn.BatchNorm2d(exp_channels)
        self.conv3=nn.Conv2d(exp_channels,out_channels,kernel_size=1,stride=1,padding=0,bias=False)
        self.bn3=nn.BatchNorm2d(out_channels)
        # ÏÈ³õÊ¼»¯Ò»¸ö¿ÕÐòÁÐ£¬Ö®ºó¸ÄÔìÆä³ÉÎª²Ð²îÁ´½Ó
        self.shortcut = nn.Sequential()
        # Ö»ÓÐ²½³¤Îª1ÇÒÊäÈëÊä³öÍ¨µÀ²»ÏàÍ¬Ê±²Å²ÉÓÃÌøÔ¾Á¬½Ó(ÏëÒ»ÏÂÌøÔ¾Á´½ÓµÄ¹ý³Ì£¬ÊäÈëÊä³öÍ¨µÀÏàÍ¬Õâ¸öÌøÔ¾Á¬½Ó¾ÍÃ»ÒâÒåÁË)
        if stride == 1 and in_channels != out_channels:
            self.shortcut = nn.Sequential(
                # ÏÂÃæµÄ²Ù×÷¾í»ý²»¸Ä±ä³ß´çºÍÍ¨µÀÊý
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self,x):
        out=self.nlin_layer(self.bn1(self.conv1(x)))
        if self.se is not None:
            out=self.bn2(self.conv2(out))
            out=self.nlin_layer(self.se(out))
        else:
            out = self.nlin_layer(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out + self.shortcut(x) if self.stride == 1 else out
        return out


class MobileNetV3_large(nn.Module):
    # (out_channels,kernel_size,exp_channels,stride,se,nl)
    cfg=[
        (16,3,16,1,False,'RE'),
        (24,3,64,2,False,'RE'),
        (24,3,72,1,False,'RE'),
        (40,5,72,2,True,'RE'),
        (40,5,120,1,True,'RE'),
        (40,5,120,1,True,'RE'),
        (80,3,240,2,False,'HS'),
        (80,3,200,1,False,'HS'),
        (80,3,184,1,False,'HS'),
        (80,3,184,1,False,'HS'),
        (112,3,480,1,True,'HS'),
        (112,3,672,1,True,'HS'),
        (160,5,672,2,True,'HS'),
        (160,5,960,1,True,'HS'),
        (160,5,960,1,True,'HS')
    ]
    def __init__(self,num_classes=5):
        super(MobileNetV3_large,self).__init__()
        self.conv1=nn.Conv2d(3,16,3,2,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(16)
        # ¸ù¾ÝcfgÊý×é×Ô¶¯Éú³ÉËùÓÐµÄBottleneck²ã
        self.layers = self._make_layers(in_channels=16)
        self.conv2=nn.Conv2d(160,960,1,stride=1,bias=False)
        self.bn2=nn.BatchNorm2d(960)
        # ¾í»ýºó²»¸úBN£¬¾ÍÓ¦¸Ã°ÑbiasÉèÖÃÎªTrue
        self.conv3=nn.Conv2d(960,1280,1,1,padding=0,bias=True)
        self.conv4=nn.Conv2d(1280,num_classes,1,stride=1,padding=0,bias=True)
        self.up1=Upsample(160,112)
        self.up2=Upsample(112,40)
        self.c1=nn.Conv2d(224,112,kernel_size=3,stride=1,padding=1)
        self.c2=nn.Conv2d(80,40,1,1)
        self.c3=nn.Conv2d(40,112,kernel_size=3,stride=2,padding=1)
        self.b4=nn.BatchNorm2d(112)
        
        self.c6=nn.Conv2d(112,160,kernel_size=3,stride=2,padding=1)
        self.b7=nn.BatchNorm2d(160)
    def _make_layers(self,in_channels):
        layers=[]
        for out_channels,kernel_size,exp_channels,stride,se,nl in self.cfg:
            layers.append(
                Bottleneck(in_channels,out_channels,kernel_size,exp_channels,stride,se,nl)
            )
            in_channels=out_channels
        return nn.Sequential(*layers)

    def forward(self,x):
        out=Hswish(self.bn1(self.conv1(x)))
        ##############################################
        out1=self.layers[:6](out)
        out2=self.layers[6:12](out1)
        out3=self.layers[12:16](out2)
        out1=out1*0.1
        out2=out2*0.2
        out3=out3*0.7
        p5=self.up1(out3)
        p4=torch.cat([p5,out2],1)
#         print(p4.size()) #14*14*224
        p4=self.c1(p4)
        p4=self.up2(p4)
#         print(p4.size())
        p3=torch.cat([p4,out1],1)
        p3=self.c2(p3)
        p3=self.c3(p3)
        p3=self.b4(p3)
        p3=Hswish(p3)
        
        p3=self.c6(p3)
        p3=self.b7(p3)
        p3=Hswish(p3)
        
        
#         print(p3.size())
        ##############################################
#         out=self.layers(out)
        
#         out=Hswish(self.bn2(self.conv2(out)))
        out=Hswish(self.bn2(self.conv2(p3)))
        out=F.avg_pool2d(out,7)
        out=Hswish(self.conv3(out))
        out=self.conv4(out)
        
        a,b=out.size(0),out.size(1)
        out=out.view(a,b)
#         return out,out1,out2,out3
        return out

In [6]:
model=MobileNetV3_large()

In [7]:
# x=torch.randn(2,3,224,224)
# y=model(x)
# print(y.size())
# print(y)

In [8]:
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader

In [9]:
import os
import random
from PIL import Image
from torch.utils.data import Dataset

random.seed(1)

class flowerDataset(Dataset):
    # 自定义Dataset类，必须继承Dataset并重写__init__和__getitem__函数
    def __init__(self, data_dir, transform=None):
        """
            花朵分类任务的Dataset
            :param data_dir: str, 数据集所在路径
            :param transform: torch.transform，数据预处理，默认不进行预处理
        """
        # data_info存储所有图片路径和标签（元组的列表），在DataLoader中通过index读取样本
        self.data_info = self.get_img_info(data_dir)
        self.transform = transform

    def __getitem__(self, index):
        path_img, label = self.data_info[index]
        # 打开图片，默认为PIL，需要转成RGB
        img = Image.open(path_img).convert('RGB')
        # 如果预处理的条件不为空，应该进行预处理操作
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.data_info)

    # 自定义方法，用于返回所有图片的路径以及标签
    @staticmethod
    def get_img_info(data_dir):
        data_info = list()
        for root, dirs, _ in os.walk(data_dir):
            # 遍历类别
            for sub_dir in dirs:
                # listdir为列出文件夹下所有文件和文件夹名
                img_names = os.listdir(os.path.join(root, sub_dir))
                # 过滤出所有后缀名为jpg的文件名（那当然也就把文件夹过滤掉了）
                img_names = list(filter(lambda x: x.endswith('.jpeg'), img_names))

                # 遍历图片
                for i in range(len(img_names)):
                    img_name = img_names[i]
                    path_img = os.path.join(root, sub_dir, img_name)
                    # 在该任务中，文件夹名等于标签名
                    label = sub_dir
                    data_info.append((path_img, int(label)))
        return data_info

In [10]:
#宏定义一些数据，如epoch数，batchsize等
MAX_EPOCH=20
BATCH_SIZE=64
LR=0.0001
log_interval=30
val_interval=1

In [11]:
# ============================ step 1/5 数据 ============================
# split_dir=os.path.join("MobileNetV3-master",".","data","splitData")
split_dir=r'/home/frz/Desktop/torchmob/'
train_dir=r'/home/fanrz/Desktop/torchmob/608'
valid_dir=os.path.join(split_dir,"val")

In [12]:
#对训练集所需要做的预处理
train_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [13]:
#对验证集所需要做的预处理
valid_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [14]:
# 构建MyDataset实例
train_data=flowerDataset(data_dir=train_dir,transform=train_transform)
valid_data=flowerDataset(data_dir=valid_dir,transform=valid_transform)

In [15]:
print(len(train_data.data_info))

5307


In [16]:
# 构建DataLoader
# 训练集数据最好打乱
# DataLoader的实质就是把数据集加上一个索引号，再返回
train_loader=DataLoader(dataset=train_data,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        drop_last=True)
valid_loader=DataLoader(dataset=valid_data,batch_size=BATCH_SIZE,drop_last=True)

In [17]:
# ============================ step 2/5 模型 ============================
if torch.cuda.is_available():
    model=nn.DataParallel(model)
    model.cuda()

In [18]:
# ============================ step 3/5 损失函数 ============================
criterion=nn.CrossEntropyLoss()
# ============================ step 4/5 优化器 ============================
optimizer=optim.Adam(model.parameters(),lr=LR, betas=(0.9, 0.99))# 选择优化器

In [19]:
# ============================ step 5/5 训练 ============================
# 记录每一次的数据，方便绘图
train_curve=list()
valid_curve=list()
model.train()
accurancy_global=0.0
for epoch in range(MAX_EPOCH):
    loss_mean=0.
    correct=0.
    total=0.
    running_loss = 0.0

    for i,data in enumerate(train_loader):
        img,label=data
        img = Variable(img)
        label = Variable(label)
        if torch.cuda.is_available():
            img=img.cuda()
            label=label.cuda()
        
#         print(img.size())
        # 前向传播
        out=model(img)
        optimizer.zero_grad()  # 归0梯度
        loss=criterion(out,label)#得到损失函数

        print_loss=loss.data.item()

        loss.backward()#反向传播
        optimizer.step()#优化
        if (i+1)%log_interval==0:
            print('epoch:{},loss:{:.4f}'.format(epoch+1,loss.data.item()))
        _, predicted = torch.max(out.data, 1)
        total += label.size(0)
#         if (i+1)%log_interval==0:
#             print("============================================")
#             print("源数据标签：",label)
#             print("============================================")
#             print("预测结果：",predicted)
#             print("相等的结果为：",predicted == label)
        correct += (predicted == label).sum()
        if (i+1)%log_interval==0:
            print(correct.item() / total)
#         print(correct.item())
    print("============================================")
    accurancy=correct.item() / total
    if accurancy>accurancy_global:
        torch.save(model.state_dict(), './weights/best321.pkl')
        print("准确率由：", accurancy_global, "上升至：", accurancy, "已更新并保存权值为weights/best.pkl")
        accurancy_global=accurancy
    print('第%d个epoch的识别准确率为：%d%%' % (epoch + 1, 100*accurancy))
torch.save(model.state_dict(), './weights/last321.pkl')
print("训练完毕，权重已保存为：weights/last.pkl")


/usr/local/lib/python3.5/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


epoch:1,loss:1.5866
0.275
epoch:1,loss:1.5029
0.2989583333333333
准确率由： 0.0 上升至： 0.29897103658536583 已更新并保存权值为weights/best.pkl
第1个epoch的识别准确率为：29%
epoch:2,loss:1.5302
0.2994791666666667
epoch:2,loss:1.4561
0.3072916666666667
准确率由： 0.29897103658536583 上升至： 0.3094512195121951 已更新并保存权值为weights/best.pkl
第2个epoch的识别准确率为：30%
epoch:3,loss:1.3492
0.353125
epoch:3,loss:1.3255
0.346875
准确率由： 0.3094512195121951 上升至： 0.34508384146341464 已更新并保存权值为weights/best.pkl
第3个epoch的识别准确率为：34%
epoch:4,loss:1.2895
0.40208333333333335
epoch:4,loss:1.3377
0.40390625
准确率由： 0.34508384146341464 上升至： 0.4009146341463415 已更新并保存权值为weights/best.pkl
第4个epoch的识别准确率为：40%
epoch:5,loss:1.0882
0.4796875
epoch:5,loss:1.1122
0.4778645833333333
准确率由： 0.4009146341463415 上升至： 0.4717987804878049 已更新并保存权值为weights/best.pkl
第5个epoch的识别准确率为：47%
epoch:6,loss:0.8882
0.5640625
epoch:6,loss:1.1194
0.5596354166666667
准确率由： 0.4717987804878049 上升至： 0.5544969512195121 已更新并保存权值为weights/best.pkl
第6个epoch的识别准确率为：55%
epoch:7,loss:0.7720
0.663541666

In [20]:
torch.__version__

'1.0.0'